In [178]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import ykey
import json
import numpy as np

In [179]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [180]:
Base.prepare(engine, reflect=True)

In [181]:
session = Session(engine)

In [138]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, name, latitude, longitude, street, city FROM breweries", con=engine)
brew_df.set_index('id')

,name,latitude,longitude,street,city
id,,,,,
5016,Bark Brewing Company,36.0620103554184,-79.8436043406271,3021 Spring Garden St,Greensboro
5033,Blowing Rock Brewing Company,36.1327196,-81.677229,152 Sunset Dr,Blowing Rock
5035,Blue Ghost Brewing Company,35.4368388,-82.530580304217,125 Underwood Rd,Fletcher
5067,Bull Durham Beer Co,35.9915653,-78.90481567784,409 Blackwell St,Durham
5090,Dirtbag Ales Brewery and Taproom,34.9939752576824,-78.9220969691311,3623 Legion Rd,Hope Mills
...,...,...,...,...,...
5273,Thirsty Monk Brewery,35.59410425,-82.5553147520814,92 Patton Ave,Asheville
5283,Triskelion Brewing Company,35.3206001746298,-82.4578969903109,340 7th Avenue East,Hendersonville
5293,Valley River Brewery & Eatery,35.0889040369639,-84.0352081706441,71 Tennessee St,Murphy


In [12]:
# set up API Key
headers = {'Authorization': 'Bearer %s' % ykey}

In [21]:
# set base url for API call
base_url = 'https://api.yelp.com/v3/businesses/search'

In [163]:
# make API call and store results in dictionaries

name_dict = {}
reviews_dict = {}
rating_dict = {}
img_url_dict = {}

for index, row in brew_df.iterrows():
    params = {
    'term': 'brewery',
    'latitude': row.latitude,
    'longitude': row.longitude,
    'sort_by': 'distance',
    'radius': 50
    }
    
    
    
    
    response = requests.get(base_url, params=params, headers=headers)
    json_resp = response.json()
    print(response)
    
    try:
        name_dict.update({row.id: json_resp['businesses'][0]['name']})
        reviews_dict.update({row.id: json_resp['businesses'][0]['review_count']})
        rating_dict.update({row.id: json_resp['businesses'][0]['rating']})
        img_url_dict.update({row.id: json_resp['businesses'][0]['image_url']})

    except:
        pass

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [400]>
<Response [200]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [200

In [155]:
# json.loads(response.text)
json_resp['businesses'][1]

{'id': 'dB1cQtRFQi_QsgOPvZa03w',
 'alias': 'mamacitas-taqueria-asheville',
 'name': "Mamacita's Taqueria",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/3F6psuEIZ69jl_sow6BHqA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/mamacitas-taqueria-asheville?adjust_creative=rbENiOmrenPdRGUpQbW6Yg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=rbENiOmrenPdRGUpQbW6Yg',
 'review_count': 225,
 'categories': [{'alias': 'tacos', 'title': 'Tacos'},
  {'alias': 'beerbar', 'title': 'Beer Bar'}],
 'rating': 4.0,
 'coordinates': {'latitude': 35.592007, 'longitude': -82.55114},
 'transactions': [],
 'price': '$',
 'location': {'address1': '77A Biltmore Ave',
  'address2': '',
  'address3': '',
  'city': 'Asheville',
  'zip_code': '28801',
  'country': 'US',
  'state': 'NC',
  'display_address': ['77A Biltmore Ave', 'Asheville, NC 28801']},
 'phone': '+18282558080',
 'display_phone': '(828) 255-8080',
 'distance': 37.8976085285238}

In [170]:
#combine lists into drataframe
yelp_df = pd.DataFrame({'Review Count':pd.Series(reviews_dict), 'Avg. Rating':pd.Series(rating_dict), 'Image URL':pd.Series(img_url_dict)})
yelp_df

,Review Count,Avg. Rating,Image URL
5016,101,3.5,https://s3-media2.fl.yelpcdn.com/bphoto/6k-JUm...
5033,284,4.0,https://s3-media1.fl.yelpcdn.com/bphoto/LWlXdB...
5035,47,4.5,https://s3-media2.fl.yelpcdn.com/bphoto/KlaFeN...
5105,92,4.5,https://s3-media3.fl.yelpcdn.com/bphoto/hkI8OW...
5121,5,5.0,https://s3-media1.fl.yelpcdn.com/bphoto/7-Yj76...
...,...,...,...
5273,328,4.0,https://s3-media3.fl.yelpcdn.com/bphoto/29iEaf...
5283,20,4.0,https://s3-media1.fl.yelpcdn.com/bphoto/SgzAeJ...
5293,112,3.5,https://s3-media2.fl.yelpcdn.com/bphoto/p2CiNa...
5304,87,4.0,https://s3-media2.fl.yelpcdn.com/bphoto/Unoog8...


In [171]:
# prepare df for merge
yelp_df.rename(columns = {'index':'ID'})
yelp_df.index.name = 'id'
yelp_df

,Review Count,Avg. Rating,Image URL
id,,,
5016,101,3.5,https://s3-media2.fl.yelpcdn.com/bphoto/6k-JUm...
5033,284,4.0,https://s3-media1.fl.yelpcdn.com/bphoto/LWlXdB...
5035,47,4.5,https://s3-media2.fl.yelpcdn.com/bphoto/KlaFeN...
5105,92,4.5,https://s3-media3.fl.yelpcdn.com/bphoto/hkI8OW...
5121,5,5.0,https://s3-media1.fl.yelpcdn.com/bphoto/7-Yj76...
...,...,...,...
5273,328,4.0,https://s3-media3.fl.yelpcdn.com/bphoto/29iEaf...
5283,20,4.0,https://s3-media1.fl.yelpcdn.com/bphoto/SgzAeJ...
5293,112,3.5,https://s3-media2.fl.yelpcdn.com/bphoto/p2CiNa...


In [172]:
#merge dataframes on 'id'
review_brew_df = pd.merge(yelp_df, brew_df, on='id', how='outer')
review_brew_df

,id,Review Count,Avg. Rating,Image URL,name,latitude,longitude,street,city
0,5016,101.0,3.5,https://s3-media2.fl.yelpcdn.com/bphoto/6k-JUm...,Bark Brewing Company,36.0620103554184,-79.8436043406271,3021 Spring Garden St,Greensboro
1,5033,284.0,4.0,https://s3-media1.fl.yelpcdn.com/bphoto/LWlXdB...,Blowing Rock Brewing Company,36.1327196,-81.677229,152 Sunset Dr,Blowing Rock
2,5035,47.0,4.5,https://s3-media2.fl.yelpcdn.com/bphoto/KlaFeN...,Blue Ghost Brewing Company,35.4368388,-82.530580304217,125 Underwood Rd,Fletcher
3,5105,92.0,4.5,https://s3-media3.fl.yelpcdn.com/bphoto/hkI8OW...,Flytrap Brewing,34.24034335,-77.9454578768839,319 Walnut St,Wilmington
4,5121,5.0,5.0,https://s3-media1.fl.yelpcdn.com/bphoto/7-Yj76...,Ghost Harbor Brewing Company,36.3004222857143,-76.2187962244898,606 B E Colonial Ave,Elizabeth City
...,...,...,...,...,...,...,...,...,...
265,5209,NaN,NaN,NaN,Northern Outer Banks Brewing Company,None,None,520 Old Stoney Rd,Corolla
266,5221,NaN,NaN,NaN,Oyster House Brewing Co.,None,None,625 Haywood Rd,Asheville
267,5258,NaN,NaN,NaN,Steel String Craft Brewery,None,None,106 S Greensboro St Ste A,Carrboro
268,5316,NaN,NaN,NaN,York Chester Brewing Company,None,None,1500 River Dr Ste 250,Belmont


In [182]:
# add data to SQLite database
review_brew_df.to_sql('yelp_breweries', con=engine)

ValueError: Table 'yelp_breweries' already exists.

In [183]:
#test sqlite data
engine.execute("SELECT * FROM yelp_breweries").fetchall()[1]

(1, 5033, 284.0, 4.0, 'https://s3-media1.fl.yelpcdn.com/bphoto/LWlXdB6wDC9qJavOOixi4A/o.jpg', 'Blowing Rock Brewing Company', '36.1327196', '-81.677229', '152 Sunset Dr', 'Blowing Rock')

In [184]:
review_brew_df.to_csv('yelp_reviews.csv')